### Import Packages

In [ ]:
import argparse
import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
from skimage.viewer import ImageViewer
import sys
sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque
global score_list
import json
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD , Adam
import tensorflow as tf

### Step 5: Checking the performance of code from GitHub

In [ ]:
actions = 2
gamma = 0.99
observation = 1000
explore = 3000000
final_epsilon = 0.0001
inital_epsilon = 0.1
replay_memory = 50000 
batch = 32 
frame_per_actie = 1
learning_rate = 1e-4

img_rows , img_cols, img_channels = 80, 80, 4

global losslist
scorelist = []
losslist = []

def buildmodel()
    model = Sequential()
    model.add(Convolution2D(32, kernel_size = (8, 8), strides=(4, 4), padding='same',input_shape=(img_rows,img_cols,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, kernel_size = (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, kernel_size = (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.load_weights("model.h5")
    adam = Adam(lr=learning_rate)
    model.compile(loss='mse',optimizer=adam)
    return model

def trainNetwork(model,args):
    global scorelist
    global losslist
    global s_t
    global x_t
    global x_t1
    global s_t1
    
    game_state = game.GameState()

    #Store the previous observations in replay memory
    replaymemory = deque()

    #Get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(acties)
    do_nothing[0] = 1
    image, reward, done, score = game_state.frame_step(do_nothing)

    image = skimage.color.rgb2gray(image)
    image = skimage.transform.resize(image,(80,80))
    image = skimage.exposure.rescale_intensity(image, out_range=(0,255))
    image = image / 255.0
    reshaped_image = np.stack((image, image, image, image), axis=2)
    reshaped_image = reshaped_image.reshape(1, reshaped_image.shape[0], reshaped_image.shape[1], reshaped_image.shape[2])  #1*80*80*4

    #Load model weights, there will be no training
    if args['mode'] == 'Run':
        observe = 999999999    
        epsilon = final_epsion
        model.load_weights("model.h5")
        adam = Adam(lr=learning_rate)
        model.compile(loss='mse',optimizer=adam)
    #Training
    else:                       
        observe = observation
        epsilon = inital_epsilon

    game = 0
    while game < 1000000:
        loss = 0
        Q_state = 0
        action_index = 0
        next_reward = 0
        action = np.zeros([actions])
        #Choose an action epsilon greedy
        if game % frame_per_actie == 0:
            if random.random() <= epsilon:
                action_index = random.randrange(actions)
                action[action_index] = 1
            else:
                q = model.predict(reshaped_image)       
                max_Q = np.argmax(q)
                action_index = max_Q
                action[max_Q] = 1

        #We reduced the epsilon gradually
        if epsilon > final_epsilon and game > observe:
            epsilon -= (intial_epsilon - final_epsilon) / explore

        #Run the selected action and observed next state and reward
        next_image, next_reward, done, score = game_state.frame_step(a_t)
        
        next_image = skimage.color.rgb2gray(next_image)
        next_image = skimage.transform.resize(next_image,(80,80))
        next_image = skimage.exposure.rescale_intensity(next_image, out_range=(0, 255))
        next_image = next_image / 255.0
        next_image = next_image.reshape(1, next_image.shape[0], next_image.shape[1], 1) #1x80x80x1
        reshaped_image-next = np.append(next_image, reshaped_image[:, :, :, :3], axis=3)

        #Store the transition in D
        replaymemory.append((reshaped_image, action_index, next_reward, reshaped_image_next, done))
        if len(replaymemory) > replay_memory:
            replaymemory.popleft()

        #Only train if done observing
        if t > observe:
            #Sample a minibatch to train on
            minibatch = random.sample(replaymemory, batch)

            #Now we do the experience replay
            state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
            state_t = np.concatenate(state_t)
            state_t1 = np.concatenate(state_t1)
            targets = model.predict(state_t)
            Q_state = model.predict(state_t1)
            targets[range(BATCH), action_t] = reward_t + GAMMA*np.max(Q_state, axis=1)*np.invert(terminal)

            loss += model.train_on_batch(state_t, targets)

        reshaped_image = reshaped_image_next
        step = step + 1

        #Save progress every 10000 iterations
        if step % 1000 == 0:
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
                
        losslist.append(loss)
        
        if done:
            scorelist.append(score)
        state = ""
        if step <= observe:
            state = "observe"
        elif t > observe and t <= observe + explore:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", step, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", next_reward, \
            "/ Q_MAX " , np.max(Q_state), "/ Loss ", loss, "/ Score", score)

def playGame(args):
    model = buildmodel()
    trainNetwork(model,args)

def main():
    args = {'mode' : 'Train'}
    playGame(args)

In [ ]:
main()